In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

import numpy as np
np.random.seed(42)

import tensorflow as tf
tf.random.set_seed(42)

import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

#from utils import confusion_matrix



In [9]:
csv_filename = '/content/drive/MyDrive/motionProject/2023-02-13_6axis_data.csv'
df = pd.read_csv(csv_filename, encoding='utf8', low_memory=False)
df.shape


(2631, 8)

In [15]:
df.head()

,dataId,label,accX,accY,accZ,gyroX,gyroY,gyroZ
0,1,LeftRight,2.8379,2.9480,8.7386,42.5609,32.2639,105.0292
1,1,LeftRight,2.8283,2.5077,9.0018,22.6534,107.0886,177.1081
2,1,LeftRight,2.8283,2.5603,9.6335,48.0526,126.3097,179.8540
3,1,LeftRight,2.9001,2.8235,9.9254,131.1149,115.3262,182.5998
4,1,LeftRight,3.0389,4.1539,9.6861,194.2697,155.1412,222.4148


In [14]:
df.tail()

,dataId,label,accX,accY,accZ,gyroX,gyroY,gyroZ
2626,50,LeftRight,1.8664,10.6672,12.4379,42.5609,413.9387,-204.5667
2627,50,LeftRight,1.5266,4.5224,14.2182,78.2571,-213.4908,-372.0643
2628,50,LeftRight,1.8712,6.3745,9.4851,-2.7459,-218.2960,-210.0584
2629,50,LeftRight,2.2971,4.8191,9.9206,-135.2337,-43.9338,-5.4917
2630,50,LeftRight,2.6369,3.2112,10.2796,32.2639,-186.7186,21.9669


In [16]:
epochs = 30
batch_size = 16
n_hidden = 32

In [17]:
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [21]:
from sklearn.model_selection import train_test_split

X_train, Y_train = train_test_split(df, test_size=0.33, random_state=42)
# train-test data split
# X는 input, Y는 (정답)output

In [22]:
X_train.shape

(1762, 8)

In [23]:
Y_train.shape

(869, 8)

In [24]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])

# 분류하고 싶은 클래스 개수
n_classes = _count_classes(Y_train) 

model = Sequential()

# n_hidden : 은닉계층에 임베딩되는 차원
# LSTM cell =  n_hidden 차원으로 만드는 것
# timestep : split해서 넣을 데이터 절편 크기
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# input차원 - (timesteps, input_dim)

model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

# loss : predicted 결과와 real 결과의 차이
# optimizer : local optimum에 빠지지 않도록 넘어가게 해주는 장치, Adam 많이 씀
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

# Evaluate
print(confusion_matrix(Y_test, model.predict(X_test)))

KeyError: ignored